# Continuum Data
How does the continuum data fit into our data parsing/storage model that we've implemented thus far?

In [1]:
import os

In [2]:
chianti_dbase = '/Users/willbarnes/Documents/work/ssw/chianti/'

In [22]:
with open(os.path.join(chianti_dbase,'continuum','heseq_2photon.dat'),'r') as f:
    lines = f.readlines()

In [26]:
lines[1].split()

['2',
 '5.094e+01',
 '0.000',
 '0.153',
 '0.495',
 '0.854',
 '1.176',
 '1.457',
 '1.696',
 '1.902',
 '2.081',
 '2.218',
 '2.356',
 '2.454',
 '2.552',
 '2.631',
 '2.689',
 '2.748',
 '2.788',
 '2.807',
 '2.846',
 '2.846',
 '2.846',
 '2.846',
 '2.846',
 '2.807',
 '2.788',
 '2.748',
 '2.689',
 '2.631',
 '2.552',
 '2.454',
 '2.356',
 '2.218',
 '2.081',
 '1.902',
 '1.696',
 '1.457',
 '1.176',
 '0.854',
 '0.495',
 '0.153',
 '0.000']

In [16]:
lines[0].split()

['6', '41']

In [8]:
%%bash 
tail -n 10 /Users/willbarnes/Documents/work/ssw/chianti/continuum/gffgu.dat

  10000.00      1584.893      1.076466    
  10000.00      1995.262      1.080763    
  10000.00      2511.886      1.085136    
  10000.00      3162.278      1.089429    
  10000.00      3981.072      1.093450    
  10000.00      5011.872      1.097171    
  10000.00      6309.573      1.100496    
  10000.00      7943.282      1.102878    
  10000.00      10000.00      1.103985    
------------------------------------------
